In [26]:
import sys  
!{sys.executable} -m pip install textblob 

  Using cached https://files.pythonhosted.org/packages/60/f0/1d9bfcc8ee6b83472ec571406bd0dd51c0e6330ff1a51b2d29861d389e85/textblob-0.15.3-py2.py3-none-any.whl


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [37]:
from IPython.display import display
import timeit
from collections import defaultdict
import math
import numpy as np
import pandas as pd
import random
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.dates as md
%matplotlib inline
import operator 

from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

from bs4 import BeautifulSoup
import re
import nltk
from nltk import word_tokenize
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk import sentiment
# from autocorrect import Speller # For spelling correction
# from autocorrect import spell
from urllib import request
from textblob import TextBlob

Load WordNet

In [38]:
url_pos = r'https://raw.githubusercontent.com/jeffreybreen/twitter-sentiment-analysis-tutorial-201107/master/data/opinion-lexicon-English/positive-words.txt'

url_neg = r'https://raw.githubusercontent.com/jeffreybreen/twitter-sentiment-analysis-tutorial-201107/master/data/opinion-lexicon-English/negative-words.txt'

pos_list = request.urlopen(url_pos).read().decode('utf-8')[1:]
pos_list = pos_list[pos_list.find("a+"):].split("\n")

neg_list = request.urlopen(url_neg).read().decode('ISO-8859-1')[1:]
neg_list = neg_list[neg_list.find("2-faced"):].split("\n")

In [39]:
df = pd.read_csv('../reference/Amazon_Unlocked_Mobile.csv', delimiter = ",")
n = len(df)
df.columns = ['Product', 'Brand', 'Price', 'Rating', 'Review', 'Votes']
df['id_col'] = range(0, n)
df['Label'] = 0
df.loc[df['Rating'] > 3, 'Label'] = 1
df.loc[df['Rating'] < 3, 'Label'] = -1
# n_reviews = 100 # Let's get a sample
# keep = sorted(random.sample(range(1,n),n_reviews))
# keep += list(set(test.review_id)) # this are the reviews annotated for test

# df = df[df.id_col.isin(keep)]
n_reviews = len(df)
df['id_new_col'] = range(0, n_reviews)

df.head()

,Product,Brand,Price,Rating,Review,Votes,id_col,Label,id_new_col
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0,0,1,0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0,1,1,1
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0,2,1,2
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0,3,1,3
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0,4,1,4


In [40]:
len(df)

413840

In [41]:
id_prod = 69

for val in df[df.id_new_col == id_prod].Review:
    print(val)

Nokia Asha 302 Unlocked GSM Phone with 3.2MP Camera, Video, QWERTYDependableTraditional Nokia Menu'sNot Complicated like 'Smart Phones'DurableEasy to use on Straighttalk, Internet, WiFi, Bluetooth.


In [42]:
def get_tokens(df, stem = False, negation = False):
    stemmer = PorterStemmer()
    stop = set(stopwords.words('english'))
    reviews = []  
    i = 1
    
    for review in df["Review"]:
        tokenized_review = []      

        review = str(review).lower() # lowercase
        
        # Remove every character except A-Z, a-z,space 
        # and punctuation (we'll need it for negation)
        review = re.sub(r'[^A-Za-z /.]','',review) 
        
        # mark_negation needs punctuation separated by white space.
        review = review.replace(".", " .")   
        
        tokens = word_tokenize(review)
        
        
        for token in tokens:
            # Remove single characters and stop words
            if (len(token)>1 or token == ".") and token not in stop: 
                if stem:
                    tokenized_review.append(stemmer.stem(get_synonym(token)))            
                else:
                    tokenized_review.append(get_synonym(token))
        
        if negation:
            tokenized_review = sentiment.util.mark_negation(tokenized_review)   
        
        # Now we can get rid of punctuation and also let's fix some spellings:
        tokenized_review = [correction(x) for x in tokenized_review if x != "." ]
        
            
        reviews.append(tokenized_review)
        
        if i%100 == 0:
            print('progress: ', (i/len(df["Review"]))*100, "%")
        i = i + 1
        
    return reviews
 

def get_pos(tokenized_reviews):
    tokenized_pos = []
    
    for review in tokenized_reviews:
        tokenized_pos.append(nltk.pos_tag(review))
    
    return tokenized_pos
        
    
def get_frequency(tokens):    
    term_freqs = defaultdict(int)    
    
    for token in tokens:
        term_freqs[token] += 1 
            
    return term_freqs


def get_tdm(tokenized_reviews):
    tdm = []
    
    for tokens in tokenized_reviews:
        tdm.append(get_frequency(tokens))
    
    return tdm

def normalize_tdm(tdm):    
    tdm_normalized = []
        
    for review in tdm:
        den = 0
        review_normalized = defaultdict(int)
        
        for k,v in review.items():
            den += v**2
        den = math.sqrt(den)
    
        for k,v in review.items():
            review_normalized[k] = v/den
        
        tdm_normalized.append(review_normalized)
        
    return tdm_normalized

def get_all_terms(tokenized_reviews):
    all_terms = []
    
    for tokens in tokenized_reviews:
        for token in tokens:
            all_terms.append(token)
            
    return(set(all_terms))
    
def get_all_terms_dft(tokenized_reviews, all_terms):
    terms_dft = defaultdict(int)  
    
    for term in all_terms: 
        for review in tokenized_reviews:
            if term in review:
                terms_dft[term] += 1
                
    return terms_dft


def get_tf_idf_transform(tokenized_reviews, tdm, n_reviews):
    tf_idf = []        
    all_terms = get_all_terms(tokenized_reviews)    
    terms_dft = get_all_terms_dft(tokenized_reviews, all_terms)
    
    for review in tdm:
        review_tf_idf = defaultdict(int)
        for k,v in review.items():
            review_tf_idf[k] = v * math.log(n_reviews / terms_dft[k], 2)
        
        tf_idf.append(review_tf_idf)     
    
    return tf_idf


def get_idf_transform(tokenized_reviews, tdm, n_reviews):
    idf = []    
    terms_dft = defaultdict(int)    
    
    all_terms = get_all_terms(tokenized_reviews)
    
    for term in all_terms: 
        for review in tokenized_reviews:
            if term in review:
                terms_dft[term] += 1
    
    for review in tdm:
        review_idf = defaultdict(int)
        for k,v in review.items():
            review_idf[k] = math.log(n_reviews / terms_dft[k], 2)
        
        idf.append(review_idf)     
    
    return idf


def correction(x):
    ok_words = ["microsd"]
    
    if x.find("_NEG") == -1 and x not in ok_words: # Don't correct if they are negated words or exceptions
        b = TextBlob(x) 
#         spell = Speller(lang='en')
        return str(b.correct())
    else:
        return x

def get_synonym(word):
    synonyms = [["camera","video", "display"], 
                ["phone", "cellphone", "smartphone", "phones"],
               ["setting", "settings"],
               ["feature", "features"],
               ["pictures", "photos"],
               ["speakers", "speaker"]]
    synonyms_parent = ["camera", "phone", "settings", "features", "photos", "speakers"]
    
    for i in range(len(synonyms)):
        if word in synonyms[i]:
            return synonyms_parent[i]
    
    return word


def get_similarity_matrix(similarity, tokenized_reviews):
    similarity_matrix = []
    all_terms = get_all_terms(tokenized_reviews)
    
    for review in similarity:
        similarity_matrix_row = []
        for term in all_terms:
            similarity_matrix_row.append(review[term])
            
        similarity_matrix.append(similarity_matrix_row)
            
    return similarity_matrix

In [43]:
tic=timeit.default_timer()

tokenized_reviews = get_tokens(df, stem = False, negation = False)
tokenized_pos = get_pos(tokenized_reviews)
tdm = get_tdm(tokenized_reviews)
vsm = normalize_tdm(tdm)
tf_idf = get_tf_idf_transform(tokenized_reviews, tdm, n_reviews)

toc=timeit.default_timer()

print("minutes: ", (toc - tic)/60)

progress:  0.02416392808815001 %
progress:  0.04832785617630002 %
progress:  0.07249178426445002 %
progress:  0.09665571235260004 %
progress:  0.12081964044075005 %
progress:  0.14498356852890004 %
progress:  0.16914749661705006 %
progress:  0.19331142470520007 %
progress:  0.2174753527933501 %
progress:  0.2416392808815001 %
progress:  0.2658032089696501 %
progress:  0.2899671370578001 %
progress:  0.31413106514595013 %
progress:  0.3382949932341001 %
progress:  0.36245892132225016 %
progress:  0.38662284941040015 %
progress:  0.41078677749855014 %
progress:  0.4349507055867002 %
progress:  0.45911463367485017 %
progress:  0.4832785617630002 %
progress:  0.5074424898511503 %
progress:  0.5316064179393002 %
progress:  0.5557703460274502 %
progress:  0.5799342741156002 %
progress:  0.6040982022037502 %
progress:  0.6282621302919003 %
progress:  0.6524260583800502 %
progress:  0.6765899864682002 %
progress:  0.7007539145563503 %
progress:  0.7249178426445003 %
progress:  0.74908177073265

KeyboardInterrupt: 

In [104]:
lookup_review = 3
for val in df[df.id_new_col == lookup_review]["Review"]: print(val)
display(tokenized_reviews[lookup_review])
# display(tokenized_pos[lookup_review])
# display(tdm[lookup_review])
# display(tf_idf[lookup_review])

The phone never worked


['phone', 'never', 'worked']

'phone never worked'

[('phone', 'NN'), ('never', 'RB'), ('worked', 'VBD')]

defaultdict(int, {'phone': 1, 'never': 1, 'worked': 1})

defaultdict(int,
            {'phone': 0.9573556625915063,
             'never': 5.107803289534515,
             'worked': 4.965784284662088})

In [119]:
import nltk.data
from gensim.models import word2vec, KeyedVectors

def generate_z1(tokenized_reviews,num_workers = 4, model_name = "model1_100features_40minwords_5context"):
    """Set values for the parameters of the your word2vec model and train it on the extracted sentences from both the train and unlabeled_train data and
      generate the collection of these representations Z1.
      You should carry out the following.
      1) Set the parameters as mentioned below. 
        A)Constrained Paramters : "context length", "embedding dimension", "min_words" (Please check the question for the values.)
        B)Optional Parameters: "number of workers", "downsample setting"
      2) Train your word2vec model and save it.
      3) Store the collection of word embeddings and the word_list(z1 and word_list_z1) .

      Arg: sentences: List of tokenized sentences (List)
            num_features: Word vector dimensionality (int)
            min_word_count: Minimum word count (int)
            context: Context window size (int)
            num_workers: Number of threads to run in parallel (int)
            downsampling: Downsample setting for frequent words (float)
            model_name = Name to save your model (str)
      Returns:
            trained_word2vec_model: word2vec model trained on the tokenized sentences.
            z1: word embeddings (ndarray)
            word_list_z1: List of tokens in the model (List)

      

    """
    print("Training model...")
    ### Add your code here.
    trained_word2vec_model = word2vec.Word2Vec(tokenized_reviews, workers=num_workers)

    trained_word2vec_model.init_sims(replace=True)
    trained_word2vec_model.save(model_name)
    z1 = trained_word2vec_model.wv.vectors
    word_list_z1 = trained_word2vec_model.wv.index2word
    #######################
    return trained_word2vec_model, z1, word_list_z1

# Set values for the parameters of the your word2vec model and train it on the extracted sentences from both the train and unlabeled_train data and
# generate the collection of these representations Z1.
# Add the function parameter values below.
### Add your code here.
num_features = 10
# min_word_count = 40
# context = 5
######################

model1, z1, word_list_z1 = generate_z1(tokenized_reviews,num_features)

Training model...


In [120]:
len(z1)

801

In [132]:
def fit_kmeans(z,word_list_z,num_clusters = 20):
    """ Fit kmeans on the embedding representations and return a mapping of word to cluster indices. Please use the default values for
        the rest of the kmeans parameters.

        Arg: z: word embeddings (ndarray)
              word_list_z: List of tokens in the model (List) 
              num_clusters: Number of clusters (int)
        Returns:
            pre_trained_word2vec_model: word2vec model trained on the tokenized sentences.
            z2: word embeddings (ndarray)
            word_centroid_map_z: A mapping of word to cluster index it belongs to. (Dict)      

    """
    ### Add your code here.
    kmeans = KMeans(n_clusters=num_clusters)
    z_cluster_result = kmeans.fit_predict(z)
    n = len(word_list_z)
    word_centroid_map_z = {}
    for i in range(n):
        word_centroid_map_z[word_list_z[i]] = z_cluster_result[i] 
    ######################

    return word_centroid_map_z


word_centroid_map_z1 = fit_kmeans(z1, word_list_z1, 20)

In [133]:
def print_clusters(word_centroid_map_z, model_name):
    """ Print min(20, cluster_size) words for each of the clusters.

        Args: word_centroid_map_z: A mapping of word to cluster index it belongs to. (Dict)  
              model_name: Model Name (str)


    """
    print("The clusters for {0} are....".format(model_name))
    ### Add your code here.
    for i in range(20):
        cluster = i+1
        print("\nCluster %d" % cluster)
        
        words = []
        count = 0
        for key, value in word_centroid_map_z.items():
            if count == 20:
                    break
            if value == i:
                words.append(key)
                count += 1
        print (words)
    ######################
print_clusters(word_centroid_map_z1, "model1")

The clusters for model1 are....

Cluster 1
['excelente', 'charges', 'updated', 'randomly', 'updates', 'contract']

Cluster 2
['waiting']

Cluster 3
['freeze', 'mind', 'parents', 'cracked']

Cluster 4
['seconds']

Cluster 5
['wanted', 'model', 'hand', 'color', 'exactly', 'looking', 'fake', 'cool', 'number', 'cover', 'maps', 'live', 'kept', 'maybe', 'gets', 'mode', 'access', 'difference', 'cable', 'says']

Cluster 6
['language']

Cluster 7
['birthday']

Cluster 8
['clot']

Cluster 9
['huge']

Cluster 10
['drop']

Cluster 11
['cel']

Cluster 12
['reception']

Cluster 13
['wonderful']

Cluster 14
['high', 'anymore', 'died', 'reset', 'connected', 'turned', 'longer', 'complete', 'lasts', 'figure', 'functions', 'friends', 'anyway', 'mhz', 'faster', 'impressive', 'budget', '.for', 'bright', 'standard']

Cluster 15
['phone', 'good', 'great', 'like', 'one', 'battery', 'would', 'camera', 'screen', 'use', 'get', 'works', 'love', 'new', 'excellent', 'work', 'price', 'buy', 'dont', 'well']

Cluster 

In [134]:
def create_bag_of_centroids(tokenized_reviews, word_centroid_map, num_clusters = 20):
    """ Create a bag of kmeans centroids for each review i.e. for a review we return an array of length num_clusters with each 
        element of the array indicating how many words(tokens) of the review belong to that cluster.

        Args: tokenized_review: list of tokens corresponding to a review (List)
              word_centroid_map: word to cluster_index map for the model (Dict)
              num_clusters: Number of clusters (int)
        
        Returns: bag_of_centroids: An array containing the count of tokens in each cluster (ndarray)
    """
  
    ### Add your code here.
    bag_of_centroids = np.zeros(num_clusters)
    n = len(tokenized_reviews)
    for word in tokenized_reviews:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #######################
    return bag_of_centroids

In [135]:
def create_design_matrices(data, tokenized_reviews, word_centroid_map_z1, num_clusters = 20):
    """ Creates the design matrices X1(for trained model) and X2(for pretrained google word2vec model) for the given data

        Args: data: Train/Test data (pandas.core.frame.DataFrame)
              cleaned_reviews: List of tokenized reviews(sentences are not split and stopwords removed) (List)
              word_centroid_map_z1: word to cluster map for model1 (Dict)
              word_centroid_map_z2: word to cluster map for model2 (Dict)
              num_clusters: Number of KMeans clusters (int)

        Returns:
              x1_data: Design matrices X1-- Shape should be num_reviews*num_clusters (np.ndarray) 
              x2_data: Design matrices X2-- Shape should be num_reviews*num_clusters (np.ndarray)


    """
    ### Add your code here.
    x1_data = np.zeros((df["Review"].size, num_clusters))

    for i in range(len(tokenized_reviews)):
        x1_data[i] = create_bag_of_centroids(tokenized_reviews[i], word_centroid_map_z1, num_clusters = 20)
    ######################
    return x1_data
x1_data = create_design_matrices(df,tokenized_reviews,word_centroid_map_z1,20)

**x1_data** is the data we are going to pass in for classification.

In [66]:
def get_product_tokens(df):
    stop = set(stopwords.words('english'))
    products = []
    i = 1
    
    for product in df["Product"]:
        tokenized_product = []      

        product = product.lower() # lowercase
        
        # Remove every character except A-Z, a-z,space 
        # and punctuation (we'll need it for negation)
        product = re.sub(r'[^0-9A-Za-z \.]','',product)    
    
        # Only consider first 10 words of the product names
        tokens = word_tokenize(product)[:11]
        
        for token in tokens:
            # Remove stop words
            if token not in stop:
                tokenized_product.append(token)       
            
        products.append(tokenized_product)
        
        if i%100 == 0:
            print('progress: ', (i/len(df["Product"]))*100, "%")
        i = i + 1
        
    return products

In [67]:
tokenized_products = get_product_tokens(df)
products_tokenized_pos = get_pos(tokenized_products)
products_tdm = get_tdm(tokenized_products)
products_tf_idf = get_tf_idf_transform(tokenized_products, products_tdm, n_reviews)
products_idf = get_idf_transform(tokenized_products, products_tdm, n_reviews)

progress:  10.0 %
progress:  20.0 %
progress:  30.0 %
progress:  40.0 %
progress:  50.0 %
progress:  60.0 %
progress:  70.0 %
progress:  80.0 %
progress:  90.0 %
progress:  100.0 %


In [68]:
lookup_product = 53
display(df[df.id_new_col== lookup_product]["Product"])

# we want to grab those with higher scores (least common terms)
display(sorted(products_idf[lookup_product].items(), 
               key=operator.itemgetter(1), reverse = True)) 

# Unfortunately we can't filter through POS
display(products_tokenized_pos[lookup_product])

18416    Apple iPhone 5 16GB - Unlocked - Black (Certif...
Name: Product, dtype: object

[('refurbished', 5.442222328605075),
 ('5', 4.608232280044003),
 ('certified', 4.50635266602479),
 ('black', 2.7563309190331373),
 ('16gb', 2.6529013293777317),
 ('apple', 2.2378638300988882),
 ('iphone', 2.1778817252706553),
 ('unlocked', 0.5104570643575262)]

[('apple', 'NN'),
 ('iphone', 'NN'),
 ('5', 'CD'),
 ('16gb', 'CD'),
 ('unlocked', 'JJ'),
 ('black', 'JJ'),
 ('certified', 'VBN'),
 ('refurbished', 'VBD')]

In [69]:
colors = ["black", "red", "blue", "white", "gray", "green","yellow", "pink", "gold"]
common_terms = ["smarthphone", "phone", "cellphone", "retail", "warranty", 
                "silver", "bluetooth", "wifi", "wireless", "keyboard", "gps",
               "original", "unlocked", "camera", "certified", "international",
               "actory", "packaging", "us", "usa", "international", "refurbished", 
               "phones", "att", "verizon", "-", "8gb", "16gb", "32gb", "64gb", "contract"]

def standardize_names(products_idf, colors, common_terms):
    standard_names = []
    brands = [str(x).lower() for x in set(df.Brand)]
    
    for product in products_idf:
        
        for k, v in product.items():
            # Remove color and brand words
            if k in colors or k in common_terms or k in brands:
                product[k] = 0
        
        # Grab the first 5 words with highest score
        product = sorted(product.items(), key=operator.itemgetter(1), reverse = True)[:5]
        
        standard_names.append(product)
        
        tokenized_standard_product_names = []
        
    for product in standard_names:
        product_name = []
        for word in product:
            if word[1] > 0:
                product_name.append(word[0])

        tokenized_standard_product_names.append(product_name)
    
    
        
    return tokenized_standard_product_names

In [70]:
standard_product_names = standardize_names(products_idf, colors, common_terms)

product_tdm = get_tdm(standard_product_names)
product_vsm = normalize_tdm(product_tdm)
product_vsm[1]

defaultdict(int,
            {'mt6582m': 0.4472135954999579,
             '1331mhz': 0.4472135954999579,
             'a850': 0.4472135954999579,
             'smartphone960x540': 0.4472135954999579,
             '5.5inch': 0.4472135954999579})

In [71]:
similarity = product_tdm
product_names_clusters = int(round(n_reviews/2,0))

similarity_matrix = pd.DataFrame(get_similarity_matrix(similarity, standard_product_names), columns = get_all_terms(standard_product_names))

kmeans = KMeans(n_clusters=product_names_clusters, random_state=0).fit(similarity_matrix)
clusters=kmeans.labels_.tolist()

clustered_matrix = similarity_matrix.copy()
clustered_matrix['product_name_cluster'] = clusters
clustered_matrix['id_col'] = range(0, n_reviews)

display(clustered_matrix[:5])

count_clusters = pd.DataFrame(clustered_matrix.product_name_cluster.value_counts())
display(count_clusters[:5])


,curve,intuition,s4,large,version,6.0,4.0k,focus,space,milspec,...,cam,rogue,12mp,e5823,english,android,g3,9780,product_name_cluster,id_col
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,48,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,48,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,200,2
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,97,3
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,394,4


,product_name_cluster
16,31
22,17
25,15
51,12
20,12


In [72]:
df["cluster_name"] = list(clustered_matrix.product_name_cluster)

def create_standard_name(df):
    new_names = defaultdict(int)
    
    current_names = df.groupby('cluster_name').first().Product
    
    
    for i in set(clusters):
        cluster_name = df[df.cluster_name == i].Product.value_counts().index[0]
        new_name = []
        
        for word in cluster_name.split():
            temp_word= re.sub(r'[^0-9A-Za-z \.\-]','',word).lower()
            if temp_word not in colors and temp_word not in common_terms :
                new_name.append(word)
        new_names[i] = ' '.join(new_name)
    
    new_standard_names = []
    

    for row in df.cluster_name:
        
        new_standard_names.append(new_names[row])
    
    df["Standard_Product_Name"] = new_standard_names
    
    return df

df = create_standard_name(df)         
        
df.head()

,Product,Brand,Price,Rating,Review,Votes,id_col,Label,id_new_col,cluster_name,Standard_Product_Name
1093,5.5-Inch Unlocked Lenovo A850 3G Smartphone-(9...,NaN,161.06,5,exelente,0.0,1093,1,0,48,5.5-Inch Lenovo A850 3G Smartphone-(960x540) Q...
1164,5.5-Inch Unlocked Lenovo A850 3G Smartphone-(9...,NaN,161.06,5,Exelente,0.0,1164,1,1,48,5.5-Inch Lenovo A850 3G Smartphone-(960x540) Q...
1342,"6"" Inch Unlocked Android 4.4.2 MTK6572 Dual Co...",NaN,100.00,1,The phone never worked,1.0,1342,-1,2,200,"6"" Inch Android 4.4.2 MTK6572 Dual Core Smartp..."
1899,ALCATEL OneTouch Idol 3 Global Unlocked 4G LTE...,Alcatel,292.98,5,Iv'e had the phone for around a week now and j...,6.0,1899,1,3,97,ALCATEL OneTouch Idol 3 Global 4G LTE Smartpho...
2196,ALCATEL OneTouch Idol 3 Global Unlocked 4G LTE...,Alcatel,129.00,1,Stay away from Alcatel products. They do not d...,1.0,2196,-1,4,394,ALCATEL OneTouch Idol 3 Global 4G LTE Smartpho...
